In [ ]:
# Setting up google drive 
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# import sys
# sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

In [ ]:
# Imports
import my_utils as mu
import torch
from torch import nn
from IPython import display

In [ ]:
#Dataset and DataLoader
batch_size = 256 # Pick batch size
train_iter, test_iter = mu.load_data_fashion_mnist(batch_size) # Load Images

In [ ]:
#Stem
def stem(X):
  matrixM = [] # Initialize empty array to store tensors
  for image in X:
    # Split array in the first dimension into 4 separate arrays
    patch = torch.tensor_split(image, 4, dim=1)

    # Reshape the tesnor so all patches are ordered correctly
    new_patch = []
    for x in patch:
      new_patch.append(x.reshape(28,7))

    # Split array in the next dimension and append to array
    patches = []
    for i in range(len(new_patch)):
      for j in range(len(new_patch)):
        patches.append(torch.tensor_split(new_patch[i], 4, dim=0)[j]) 
        
    new_patches = torch.empty([0,1])
    # add each separate array to a new tensor to create a Matrix
    new_patches = torch.stack([p for p in patches])

    # flatten tensor so that each tensor is on one line
    new_patches = new_patches.flatten(1)

    # create linear layer
    convert_to_linear = nn.Linear(49, 49)
    feature_vector = convert_to_linear(new_patches)
    matrixM.append(feature_vector)
  
    # Create a tensor 
    M = torch.stack([p for p in matrixM])
  return M

In [ ]:
#Backbone
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.Linear1 = nn.Linear(num_inputs, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear1.weight)
        torch.nn.init.zeros_(self.Linear1.bias)

        self.Linear2 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear2.weight)
        torch.nn.init.zeros_(self.Linear2.bias)

        self.Linear3 = nn.Linear(49, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear3.weight)
        torch.nn.init.zeros_(self.Linear3.bias)

        self.Linear4 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear4.weight)
        torch.nn.init.zeros_(self.Linear4.bias)

        self.Linear5 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear5.weight)
        torch.nn.init.zeros_(self.Linear5.bias)

        self.Linear6 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear6.weight)
        torch.nn.init.zeros_(self.Linear6.bias)

        self.Linear7 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear7.weight)
        torch.nn.init.zeros_(self.Linear7.bias)

        self.Linear8 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear8.weight)
        torch.nn.init.zeros_(self.Linear8.bias)

        self.Linear9 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear9.weight)
        torch.nn.init.zeros_(self.Linear9.bias)

        self.Linear10 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear10.weight)
        torch.nn.init.zeros_(self.Linear10.bias)

        self.Linear11 = nn.Linear(num_hidden, num_hidden)
        torch.nn.init.kaiming_normal_(self.Linear11.weight)
        torch.nn.init.zeros_(self.Linear11.bias)

        self.Linear12 = nn.Linear(num_hidden, 10)
        torch.nn.init.kaiming_normal_(self.Linear12.weight)
        torch.nn.init.zeros_(self.Linear12.bias)
        
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, image):
        arr = []
        for x in image:
          x = torch.transpose(x, 0, 1)
          x = self.Linear1(x)
          x = self.relu(x)
          x = self.Linear2(x)

          x = torch.transpose(x, 0, 1)
          x = self.Linear3(x)
          x = self.relu(x)
          x = self.Linear4(x)

          x = torch.transpose(x, 0, 1)
          x = self.Linear9(x)
          x = self.relu(x)
          x = self.Linear10(x)

          x = torch.transpose(x, 0, 1)
          x = self.Linear11(x)
          x = self.relu(x)
          x = self.Linear12(x)

          arr.append(x)

        new_arr = torch.stack(arr, dim=0)

        x = torch.mean(new_arr, 1)
        out = F.log_softmax(x)

        return out

num_inputs, num_outputs = 16, 49
num_hidden = 32
net = Net(num_inputs, num_hidden, num_outputs)
# Initialize net with hidden layers and params

In [ ]:
#Classifier

#Loss and Optimizer
# Creare your loss here. Use Cross Entropy loss:
loss = nn.CrossEntropyLoss()
lr, wd = 0.0012, 0.0005

# COptimizer Adam with weight decay wd and learning rate lr.
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=wd)

I used the training model from lab 5 to show my model. I changed net(X) to net(stem(X)) to accomodate for my model.

In [ ]:
def accuracy(y_hat, y):  #y_hat is a matrix; 2nd dimension stores prediction scores for each class.
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1) # Predicted class is the index of max score         
    cmp = (y_hat.type(y.dtype) == y)  # because`==` is sensitive to data types
    return float(torch.sum(cmp)) # Taking the sum yields the number of correct predictions.


In [ ]:
class Accumulator:  
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n # [0, 0, ..., 0]
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    def reset(self):
        self.data = [0.0] * len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
def evaluate_accuracy(net, data_iter): 
    """Compute the accuracy for a model on a dataset."""
    metric = Accumulator(2)  # No. of correct predictions, no. of predictions
    for _, (X, y) in enumerate(data_iter):
        metric.add(accuracy(net(stem(X)), y), y.numel())
    return metric[0] / metric[1]

In [ ]:
evaluate_accuracy(net, test_iter)

In [ ]:
def train_epoch_ch3(net, train_iter, loss, optimizer, batch_size=256, num_outputs=10):  
    """The training function for one epoch."""
    # Set the model to training mode
    if isinstance(net, torch.nn.Module):
        net.train()
    # Sum of training loss, sum of training accuracy, no. of examples
    metric = Accumulator(3)
    for X, y in train_iter:
        # Compute gradients and update parameters
        y_hat = net(stem(X))
        l = loss(y_hat, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        metric.add(float(l) * len(y), accuracy(y_hat, y), y.size().numel())
    # Return training loss and training accuracy
    return metric[0] / metric[2], metric[1] / metric[2]

In [ ]:
class Animator:  
    """For plotting data in animation."""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # Incrementally plot multiple lines
        if legend is None:
            legend = []
        mu.use_svg_display()
        self.fig, self.axes = mu.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # Use a lambda function to capture arguments
        self.config_axes = lambda: mu.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # Add multiple data points into the figure
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

In [ ]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, optimizer): 
    """Train a model."""
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.1, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, optimizer)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))
    train_loss, train_acc = train_metrics

In [ ]:
num_epochs = 25
train_ch3(net, train_iter, test_iter, loss, num_epochs, optimizer)